# Artificial Intelligence
# 464/664
# Assignment #7

## General Directions for this Assignment

00. We're using a Jupyter Notebook environment (tutorial available here: https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html),
01. Output format should be exactly as requested (it is your responsibility to make sure notebook looks as expected on Gradescope),
02. Check submission deadline on Gradescope,
03. Rename the file to Last_First_assignment_7,
04. Submit your notebook (as .ipynb, not PDF) using Gradescope, and
05. Do not submit any other files.

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".

## Neural Networks: Architecture

For this assignment we will explore Neural Networks; in particular, we are going to explore model complexity. We will use the same dataset from Assignment #6 to classify a mushroom as either edible ('e') or poisonous ('p'). You are free to use PyTorch, TensorFlow, scikit-learn -- to name a few resources. The goal is to explore different model complexities (architectures) before declaring a winner. Either start with a simple network and make it more complex; or start with a complex model and pare it down. Either way, your submission should clearly demonstrate your exploration.


Your output for each model should look like the output of `cross_validate` from Assignment #6:

```
Fold: 0	Train Error: 15.38%	Validation Error: 0.00%
Fold: 1
...

Mean(Std. Dev.) over all folds:
-------------------------------
Train Error: 100.00%(0.00%) Test Error: 100.00%(0.00%)
```

Notice that "Test Error" has been replaced by "Validation Error." Split your dataset into train, test, and validation sets.


Start with a simple network. Train using the train set. Observe model's performance using the validation set.


Increase the complexity of your network. Train using the train set. Observe model's performance using the validation set.


Model complexity in Assignment #6 was depth limit. You can think of it here as the architecture of the network (number of layers and units per layer). Try at least three different network architectures.


We're trying to find a model complexity that generalizes well. (Recall high bias vs high variance discussion in class.)


Pick the network architecture that you deem best. Use the test set to report your winning model's performance. This is the ONLY time you use the test set.


Try at least three different models; more importantly, document your process: what the results were, how the winning model was determined, what was the winning model's performance on the test data. Clearly highlight these items to receive full credit.

In [8]:
# Import necessary libraries
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Load and preprocess the mushroom dataset
def load_and_preprocess_data(filepath='agaricus-lepiota.data'):
    # Define column names
    columns = ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
               'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
               'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
               'stalk-surface-below-ring', 'stalk-color-above-ring',
               'stalk-color-below-ring', 'veil-type', 'veil-color',
               'ring-number', 'ring-type', 'spore-print-color',
               'population', 'habitat']
    
    # Load data
    df = pd.read_csv(filepath, names=columns)
    
    # Separate features and target
    X = df.drop('class', axis=1)
    y = df['class'].apply(lambda x: 1 if x == 'e' else 0)  # Encode target: edible=1, poisonous=0
    
    # One-hot encode features
    X_encoded = pd.get_dummies(X)
    
    return X_encoded, y

# Split the data into train, validation, and test sets
def split_data(X, y):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

# Define a custom dense layer
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim):
        super(MyDenseLayer, self).__init__()
        self.W = self.add_weight(shape=(input_dim, output_dim), initializer='random_normal')
        self.b = self.add_weight(shape=(output_dim,), initializer='zeros')

    def call(self, inputs):
        z = tf.matmul(inputs, self.W) + self.b
        output = tf.math.sigmoid(z)
        return output

# Build a simple neural network model
def build_model(input_shape):
    model = tf.keras.Sequential([
        MyDenseLayer(input_dim=input_shape, output_dim=16),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the model
def train_and_evaluate(X_train, X_val, y_train, y_val):
    model = build_model(X_train.shape[1])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
    return model

# Main execution
X, y = load_and_preprocess_data()
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)

# Convert data to float32
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

model = train_and_evaluate(X_train, X_val, y_train, y_val)

# Evaluate on test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
test_accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Test Accuracy: {test_accuracy:.2f}")

Epoch 1/10
178/178 [==============================] - 1s 2ms/step - loss: 0.4693 - accuracy: 0.8846 - val_loss: 0.3101 - val_accuracy: 0.9139
Epoch 2/10
178/178 [==============================] - 0s 1ms/step - loss: 0.2219 - accuracy: 0.9453 - val_loss: 0.1671 - val_accuracy: 0.9549
Epoch 3/10
178/178 [==============================] - 0s 2ms/step - loss: 0.1246 - accuracy: 0.9770 - val_loss: 0.1010 - val_accuracy: 0.9803
Epoch 4/10
178/178 [==============================] - 0s 984us/step - loss: 0.0784 - accuracy: 0.9873 - val_loss: 0.0669 - val_accuracy: 0.9902
Epoch 5/10
178/178 [==============================] - 0s 1ms/step - loss: 0.0534 - accuracy: 0.9935 - val_loss: 0.0469 - val_accuracy: 0.9951
Epoch 6/10
178/178 [==============================] - 0s 2ms/step - loss: 0.0384 - accuracy: 0.9960 - val_loss: 0.0345 - val_accuracy: 0.9975
Epoch 7/10
178/178 [==============================] - 0s 1ms/step - loss: 0.0287 - accuracy: 0.9975 - val_loss: 0.0265 - val_accuracy: 0.9984
Epoc

## Experiment: Activation Function and Optimizer
Modify the 1) Activation function 2) Optimizer of any chosen model. Try at least one model for each modified component.

Explain the motivation behind the modifications you made.

Explore the effects on the performance.


In [ ]:
# Implementation and exploration.


## OPTIONAL. BONUS. Experiment: Loss Function

Modify the loss function of any chosen model.

Explain the motivation behind the modifications you made.

Explore the effects on the performance.


In [ ]:
# Implementation and exploration.

No other directions for this assignment, other than what's here and in the "General Directions" section. You have a lot of freedom with this assignment. Don't get carried away. It is expected the results may vary, being better or worse, due to the limitations of the dataset. Graders are not going to run your notebooks. The notebook will be read as a report on how different models were explored. Since you'll be using libraries, the emphasis will be on your ability to communicate your findings.

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".